In [68]:
# Cell 1: Import tất cả các thư viện cần thiết
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import sqlite3
import os
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

In [69]:
# Thiết lập CUDA_LAUNCH_BLOCKING để gỡ lỗi CUDA
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [70]:
# Định nghĩa Dataset
class M5MultiDataset(Dataset):
    def __init__(self, data, cols, window_size):
        self.data = data
        self.cols = cols
        self.win = window_size

    def __len__(self):
        return len(self.data) - self.win

    def __getitem__(self, idx):
        slice_ = self.data.iloc[idx:idx+self.win]
        X_num = torch.FloatTensor(slice_[self.cols].values)
        X_item = torch.LongTensor(slice_['item_idx'].values)
        X_store = torch.LongTensor(slice_['store_idx'].values)
        y = torch.FloatTensor([self.data['sales'].iloc[idx+self.win]]).squeeze()
        return X_num, X_item, X_store, y

In [71]:
# Định nghĩa mô hình LSTM
class LSTMEmbForecast(nn.Module):
    def __init__(self, n_items, n_stores, embed_dim, num_feats, hidden_size, num_layers=1, dropout=0.25):
        super().__init__()
        self.item_emb = nn.Embedding(n_items, embed_dim)
        self.store_emb = nn.Embedding(n_stores, embed_dim)
        input_size = embed_dim * 2 + num_feats
        self.feat_fc = nn.Linear(num_feats, num_feats * 2)
        self.lstm = nn.LSTM(input_size + num_feats, hidden_size, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, X_num, X_item, X_store):
        item_emb = self.item_emb(X_item)
        store_emb = self.store_emb(X_store)
        X_num_transformed = self.feat_fc(X_num)
        X = torch.cat((X_num_transformed, item_emb, store_emb), dim=-1)
        lstm_out, _ = self.lstm(X)
        lstm_out = self.dropout(lstm_out)
        out = self.fc(lstm_out[:, -1, :])
        return out.squeeze()

In [72]:
# Hàm huấn luyện mô hình
def train_model(model, train_loader, val_loader, num_epochs=20, lr=0.0008, device='cpu', patience=5):
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
    best_val_loss = float('inf')
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_preds, train_targets = [], []
        for X_num, X_item, X_store, y in train_loader:
            X_num, X_item, X_store, y = X_num.to(device), X_item.to(device), X_store.to(device), y.to(device)
            optimizer.zero_grad()
            out = model(X_num, X_item, X_store)
            loss = criterion(out, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            train_loss += loss.item()
            train_preds.extend(out.cpu().detach().numpy())
            train_targets.extend(y.cpu().detach().numpy())
        train_loss /= len(train_loader)
        train_mae = mean_absolute_error(train_targets, train_preds)
        train_r2 = r2_score(train_targets, train_preds)

        model.eval()
        val_loss = 0
        val_preds, val_targets = [], []
        with torch.no_grad():
            for X_num, X_item, X_store, y in val_loader:
                X_num, X_item, X_store, y = X_num.to(device), X_item.to(device), X_store.to(device), y.to(device)
                out = model(X_num, X_item, X_store)
                loss = criterion(out, y)
                val_loss += loss.item()
                val_preds.extend(out.cpu().detach().numpy())
                val_targets.extend(y.cpu().detach().numpy())
        val_loss /= len(val_loader)
        val_mae = mean_absolute_error(val_targets, val_preds)
        val_r2 = r2_score(val_targets, val_preds)

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}')
        
        # Early stopping với patience
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'model.pth')
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(f"No improvement in Val Loss for {epochs_no_improve} epoch(s).")
            if epochs_no_improve >= patience:
                print(f"Early stopping after {epoch+1} epochs...")
                break
        
        # Giảm learning rate nếu Val Loss không cải thiện
        scheduler.step(val_loss)

In [73]:
# Huấn luyện mô hình duy nhất
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Training model for all states...")
print(f"Using device: {device}")

Training model for all states...
Using device: cuda


In [74]:
# Đọc dữ liệu từ SQLite
df = pd.read_sql_query("SELECT * FROM data", sqlite3.connect('processed_data.db'))


In [75]:
# Kiểm tra giá trị của item_idx và store_idx
print("Min item_idx:", df['item_idx'].min())
print("Max item_idx:", df['item_idx'].max())
print("Min store_idx:", df['store_idx'].min())
print("Max store_idx:", df['store_idx'].max())

Min item_idx: 0
Max item_idx: 43
Min store_idx: 0
Max store_idx: 9


In [76]:
# Chia dữ liệu thành train và val
split = int(len(df) * 0.8)
df_train, df_val = df.iloc[:split], df.iloc[split:]


In [77]:
# Tạo Dataset và DataLoader
feature_cols = ['day', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos', 'week',
                'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'rolling_mean_7', 'rolling_mean_14',
                'sell_price', 'event', 'snap']
window_size = 28
batch_size = 64

train_ds = M5MultiDataset(df_train, feature_cols, window_size=window_size)
val_ds = M5MultiDataset(df_val, feature_cols, window_size=window_size)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)


In [78]:
# Khởi tạo mô hình
num_items = len(pd.read_sql_query("SELECT DISTINCT item_idx FROM data", sqlite3.connect('processed_data.db')))
num_stores = len(pd.read_sql_query("SELECT DISTINCT store_idx FROM data", sqlite3.connect('processed_data.db')))
print(f"num_items: {num_items}, num_stores: {num_stores}")

# Kiểm tra xem các giá trị có hợp lệ không
if df['item_idx'].max() >= num_items:
    raise ValueError("item_idx contains values larger than or equal to num_items!")
if df['store_idx'].max() >= num_stores:
    raise ValueError("store_idx contains values larger than or equal to num_stores!")
if df['item_idx'].min() < 0:
    raise ValueError("item_idx contains negative values!")
if df['store_idx'].min() < 0:
    raise ValueError("store_idx contains negative values!")

model = LSTMEmbForecast(
    n_items=num_items,
    n_stores=num_stores,
    embed_dim=16,
    num_feats=len(feature_cols),
    hidden_size=64,
    num_layers=2,
    dropout=0.25
).to(device)

num_items: 44, num_stores: 10


In [79]:
# Huấn luyện
train_model(model, train_loader, val_loader, num_epochs=20, device=device)
print("Model saved as model.pth")

Epoch 1/20, Train Loss: 0.0008, Val Loss: 0.0004, Train MAE: 0.0148, Val MAE: 0.0096, Train R2: 0.5797, Val R2: 0.7693
Epoch 2/20, Train Loss: 0.0005, Val Loss: 0.0005, Train MAE: 0.0110, Val MAE: 0.0101, Train R2: 0.7236, Val R2: 0.7515
No improvement in Val Loss for 1 epoch(s).
Epoch 3/20, Train Loss: 0.0005, Val Loss: 0.0003, Train MAE: 0.0105, Val MAE: 0.0082, Train R2: 0.7456, Val R2: 0.8286
Epoch 4/20, Train Loss: 0.0005, Val Loss: 0.0004, Train MAE: 0.0101, Val MAE: 0.0084, Train R2: 0.7606, Val R2: 0.7828
No improvement in Val Loss for 1 epoch(s).
Epoch 5/20, Train Loss: 0.0005, Val Loss: 0.0004, Train MAE: 0.0098, Val MAE: 0.0101, Train R2: 0.7699, Val R2: 0.7817
No improvement in Val Loss for 2 epoch(s).
Epoch 6/20, Train Loss: 0.0005, Val Loss: 0.0004, Train MAE: 0.0097, Val MAE: 0.0084, Train R2: 0.7719, Val R2: 0.7976
No improvement in Val Loss for 3 epoch(s).
Epoch 7/20, Train Loss: 0.0004, Val Loss: 0.0005, Train MAE: 0.0089, Val MAE: 0.0083, Train R2: 0.7904, Val R2: 0.